# Initialize

In [9]:
# turn on autoreload
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import os,sys
ROOT = os.path.abspath("..")          # if your notebook/ lives under project-root/notebook/
SRC  = os.path.join(ROOT, "src")
sys.path.append(SRC)

# Options Underlying

In [25]:
import pandas as pd

options_df = pd.DataFrame({
    'mult':[-100,100],
    'expiration': ['1/15/27','1/15/27'],
    'strike':[250.0,250.0],
    'spot':[252.29,252.29],
    'cp':['P','C'],
    'ea':['A','A'],
    'texp':[1.2416,1.2416],
    'vol':['27.45%','27.45%'],
    'borrow':[-0.006,-0.0060]
})

dividends_df = pd.DataFrame({
    'date':['11/8/25','2/8/26','5/10/26','8/9/26','11/8/26','2/8/27','5/8/27','8/8/27'],
    'amount':[0.26,0.26,0.26,0.26,0.26,0.40,0.40,0.40]
})

str_underlying='AAPL'
pd_val_date=pd.Timestamp('10/17/2025') 

In [26]:
REQUIRED_OPT_COLS = {"expiration","strike","spot","cp","vol"}
def assert_schema(df, required):
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {sorted(missing)}")

assert_schema(options_df, REQUIRED_OPT_COLS)
assert_schema(dividends_df, {"date","amount"})

In [13]:
from listed_pricer_data_loader import get_ivol_yc, apply_ivol_yc, fetch_and_pick
from listed_pricer_qlib import price_table_with_divs_ql
from listed_pricer_utils import _to_iso_date_key


def tag_source(df, prefix, keys=("expiration","strike","cp")):
    ren = {c: f"{prefix}{c}" for c in df.columns if c not in keys}
    return df.rename(columns=ren)

def run_pipeline(pd_val_date, options_df, dividends_df, underlying):
    """
    Returns:
      df_show, combined_full, yc, mkt
    """
    # 0) Helpers ---------------------------------------------------------------
    def _as_mmddyyyy(d):
        return pd_val_date.strftime('%m/%d/%Y')

    def _as_iso(d):
        return pd_val_date.strftime('%Y-%m-%d')

    # 1) Build vol/yield context + apply to options ---------------------------
    yc = get_ivol_yc(pd_val_date)
    opts2 = apply_ivol_yc(yc, options_df)
    legs_df=opts2[['expiration','strike','cp']]
    # 2) Price with QL (readable, per-row flat r) -----------------------------
    priced_ql_ir = price_table_with_divs_ql(
        opts2, dividends_df,
        valuation_date=_as_mmddyyyy(pd_val_date),
        default_r=0.0308,
        init_vol=0.2755,
        t_grid=400, x_grid=200,
        want_greeks=True,
        compute_combo=True
    )
    priced_ql_ir["ValuationDate"] = pd_val_date

    # 3) Fetch market rows for requested legs ---------------------------------
    # Expect legs_df to have ['Expiration','Strike','CP']
    out = fetch_and_pick(
        underlying,
        trade_date=_as_iso(pd_val_date),
        legs_df=legs_df,
        snap_to_nearest=False
    )
    issues = out["issues"]
    mkt_rows = out["matched_df"]
    print(f"issues: {issues}")

    # 4) Normalize keys/cases on both sides -----------------------------------
    # priced/pricer side
    priced = priced_ql_ir.rename(columns={"CP": "cp"}).copy()


    # market side (keep only relevant columns)
    mkt = mkt_rows[['expiration','strike','cp','bid','ask','iv','openInterest',
                    'volume','delta','gamma','vega','underlying_price','optionId']].copy()
    mkt = mkt.rename(columns={"expiration": "expiration", "strike": "strike"})
    priced = tag_source(priced, "mdl_")
    mkt = tag_source(mkt, "mkt_")


    priced["Exp_key"] = priced["expiration"].map(_to_iso_date_key)
    priced["strike"] = priced["strike"].astype(float).round(2)
    priced["cp"] = priced["cp"].str.upper().str[0]

    mkt["Exp_key"] = mkt["expiration"].map(_to_iso_date_key)
    mkt["strike"] = mkt["strike"].astype(float).round(2)
    mkt["cp"] = mkt["cp"].str.upper().str[0]

    # 5) Safe merge (one-to-one expected) -------------------------------------
    combined = priced.merge(
        mkt,
        on=['Exp_key','strike','cp'],
        how='left',
        validate='one_to_one'
    ).drop(columns=['Exp_key'])

    # 6) Display subset (fix typo) --------------------------------------------
    cols_show = ["expiration","strike","mdl_spot","mdl_cp","mdl_ea","mdl_texp","mdl_vol","mdl_borrow","mkt_bid","mdl_theo","mkt_ask"]
    # Only keep columns that exist to avoid KeyErrors during development
    cols_show = [c for c in cols_show if c in combined.columns]
    df_show = combined[cols_show].copy()

    return df_show, combined, yc, mkt

In [14]:
from listed_pricer_pipeline import run_pipeline, tag_source


In [15]:
combined_subset, combined, yc, mkt = run_pipeline(pd_val_date, options_df, dividends_df, "AAPL")
#show_df=combined[["epxiration_x","strike","mdl_spot","cp","mdl_ea","mdl_texp","mdl_vol","mdl_borrow","mkt_bid","mdl_theo","mkt_ask"]]
#show_df


issues: []


In [16]:
combined[["mdl_mult","expiration_x","strike","mdl_spot","cp","mdl_ea","mdl_texp","mdl_vol","mdl_borrow","mkt_bid","mdl_theo","mkt_ask","mdl_delta","mdl_gamma","mdl_vega","mkt_delta","mkt_gamma","mkt_vega"]]


,mdl_mult,expiration_x,strike,mdl_spot,cp,mdl_ea,mdl_texp,mdl_vol,mdl_borrow,mkt_bid,mdl_theo,mkt_ask,mdl_delta,mdl_gamma,mdl_vega,mkt_delta,mkt_gamma,mkt_vega
0,-100,1/15/27,250.0,252.29,P,A,1.2416,27.45%,-0.006,24.6,24.855713,25.00,-0.396554,0.005454,1.069353,-0.401476,0.005546,1.071017
1,100,1/15/27,250.0,252.29,C,A,1.2416,27.45%,-0.006,36.7,37.273944,37.65,0.634677,0.004935,1.068930,0.618236,0.004756,1.068378


In [17]:
import numpy as np
import pandas as pd

def _min_abs_nonzero(s: pd.Series) -> float:
    s = s.astype(float).abs()
    s = s[s > 0]
    return float(s.min()) if not s.empty else 0.0

def summarize_trade_group(g: pd.DataFrame, multiplier: float = 1.0) -> pd.Series:
    # nqty = qty normalized by smallest nonzero abs(qty)
    base = _min_abs_nonzero(g["mdl_qty"])
    nqty = (g["mdl_qty"].astype(float) / base) if base else 0.0

    # convenience
    def unit(col):
        return float((nqty * g[col].astype(float)).sum()) if col in g else np.nan

    theo_unit  = unit("mdl_theo")
    delta_unit = unit("mdl_delta")
    gamma_unit = unit("mdl_gamma")
    vega_unit  = unit("mdl_vega")
    theta_unit = unit("mdl_theta")

    # split nqty into long/short magnitudes
    npos = np.clip(nqty, 0, None)           # longs (we own +)
    nneg = -np.clip(nqty, None, 0)          # positive magnitude for shorts

    # market bid/ask (if present)
    if {"mkt_bid","mkt_ask"} <= set(g.columns):
        bid = g["mkt_bid"].astype(float)
        ask = g["mkt_ask"].astype(float)

        # Liquidation (close): sell longs at bid (+), buy shorts at ask (−)
        mkt_unit_close = float((npos * bid - nneg * ask).sum())

        # Opening (open): buy longs at ask (−), sell shorts at bid (+)
        mkt_unit_open  = float((npos * (ask) - nneg * bid).sum())
    else:
        mkt_unit_close = np.nan
        mkt_unit_open  = np.nan

    # raw/portfolio aggregates (straight add; $ if you include multiplier for price)
    out = {
        "qty_scale_base": base,
        "mdl_theo":  theo_unit,
        "mdl_delta": delta_unit,
        "mdl_gamma": gamma_unit,
        "mdl_vega":  vega_unit,
        "mdl_theta": theta_unit,
        "mkt_ask": mkt_unit_open,   # credit to CLOSE one unit
        "mkt_bid":  mkt_unit_close,    # debit to OPEN one unit
        "mdl_theo_$":     float((g["qty"] * g["mdl_theo"] * multiplier).sum()) if {"qty","mdl_theo"} <= set(g.columns) else np.nan,
        "delta_raw":  float((g["qty"] * g["mdl_delta"]).sum()) if {"qty","mdl_delta"} <= set(g.columns) else np.nan,
        "gamma_raw":  float((g["qty"] * g["mdl_gamma"]).sum()) if {"qty","mdl_gamma"} <= set(g.columns) else np.nan,
        "vega_raw":   float((g["qty"] * g["mdl_vega"]).sum())  if {"qty","mdl_vega"}  <= set(g.columns) else np.nan,
        "theta_raw":  float((g["qty"] * g["mdl_theta"]).sum()) if {"qty","mdl_theta"} <= set(g.columns) else np.nan,
    }
    return pd.Series(out)

In [18]:
import numpy as np
import pandas as pd

def append_total_row(df: pd.DataFrame, multiplier: float = 1.0) -> pd.DataFrame:
    """
    Appends a TOTAL row to `df` using summarize_trade_group(df).
    Keeps only the columns common to both, fills the rest with NaN,
    and preserves the same column order.
    """
    # get the summary
    s = summarize_trade_group(df, multiplier=multiplier)

    # 1️⃣ keep only columns common to both df and summary
    common_cols = [c for c in df.columns if c in s.index]

    # 2️⃣ build total row with those values, fill missing df-only columns with NaN
    row = {col: s.get(col, np.nan) for col in df.columns}

    # 3️⃣ mark as TOTAL for clarity
    row["rowtype"] = "TOTAL" if "rowtype" in df.columns else "TOTAL"

    # 4️⃣ append and return
    out = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    return out

In [19]:
from pandas.core.frame import com
combined["mdl_qty"]=combined["mdl_mult"]

options_with_total = append_total_row(combined, multiplier=100.0)

/var/folders/tw/8dy4vcgj7h3fvzr8_jtkzs600000gn/T/ipykernel_2931/3142134925.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


In [15]:
options_with_total.columns

Index(['mdl_mult', 'expiration_x', 'strike', 'mdl_spot', 'cp', 'mdl_ea',
       'mdl_texp', 'mdl_vol', 'mdl_borrow', 'mdl_rate', 'mdl_theo',
       'mdl_delta', 'mdl_gamma', 'mdl_theta', 'mdl_vega',
       'mdl_Combo_C_minus_P', 'mdl_ValuationDate', 'expiration_y', 'mkt_bid',
       'mkt_ask', 'mkt_iv', 'mkt_openInterest', 'mkt_volume', 'mkt_delta',
       'mkt_gamma', 'mkt_vega', 'mkt_underlying_price', 'mkt_optionId', 'qty',
       'mdl_qty', 'rowtype'],
      dtype='object')

In [20]:
options_with_total[["expiration_x","strike","mdl_spot","cp","mdl_ea","mdl_texp","mdl_vol","mdl_borrow","mkt_bid","mdl_theo","mkt_ask"]]

,expiration_x,strike,mdl_spot,cp,mdl_ea,mdl_texp,mdl_vol,mdl_borrow,mkt_bid,mdl_theo,mkt_ask
0,1/15/27,250.0,252.29,P,A,1.2416,27.45%,-0.006,24.6,24.855713,25.00
1,1/15/27,250.0,252.29,C,A,1.2416,27.45%,-0.006,36.7,37.273944,37.65
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.7,12.418231,13.05


In [14]:
# ------- change tracking helpers -------

from copy import deepcopy
import pandas as pd
import numpy as np
import json, hashlib, time

STATE = {"legs_prev": None, "params_prev": None, "step": 0}

KEYS = ("expiration","strike","cp")                 # identifies a leg
NUM_COLS = ("qty","vol","borrow","spot","texp")     # compare these for changes
EPS = dict(abs=1e-9, rel=1e-6)                      # float tolerance

def _norm_keys(df: pd.DataFrame) -> pd.DataFrame:
    d = df.copy()
    d["expiration"] = pd.to_datetime(d["expiration"]).dt.date.astype(str)
    d["strike"] = d["strike"].astype(float).round(8)
    d["cp"] = d["cp"].astype(str).str.upper().str[0]
    return d

def _is_close(a, b, abs_eps=EPS["abs"], rel_eps=EPS["rel"]):
    # works on scalars; NaNs -> equal if both NaN
    if pd.isna(a) and pd.isna(b): return True
    try:
        a, b = float(a), float(b)
        return abs(a-b) <= max(abs_eps, rel_eps*max(abs(a), abs(b), 1.0))
    except Exception:
        return a == b

def diff_legs(prev: pd.DataFrame|None, cur: pd.DataFrame):
    curN = _norm_keys(cur)
    if prev is None or len(prev)==0:
        return {"added": curN.copy(), "removed": curN.iloc[0:0].copy(), "modified": curN.iloc[0:0].copy()}

    prevN = _norm_keys(prev)

    # keys as strings for set ops
    pk_prev = prevN[list(KEYS)].astype(str).agg("|".join, axis=1)
    pk_cur  = curN[list(KEYS)].astype(str).agg("|".join, axis=1)

    add_keys = set(pk_cur) - set(pk_prev)
    rem_keys = set(pk_prev) - set(pk_cur)
    com_keys = list(set(pk_cur) & set(pk_prev))

    added   = curN[pk_cur.isin(add_keys)].reset_index(drop=True)
    removed = prevN[pk_prev.isin(rem_keys)].reset_index(drop=True)

    # compare numeric cols for common keys
    prevC = prevN[pk_prev.isin(com_keys)].set_index(list(KEYS))
    curC  = curN[pk_cur.isin(com_keys)].set_index(list(KEYS))

    mods = []
    for k in curC.index:
        row_prev = prevC.loc[k]
        row_cur  = curC.loc[k]
        changed = {}
        for c in NUM_COLS:
            if c in row_cur and c in row_prev and not _is_close(row_prev[c], row_cur[c]):
                changed[c] = (row_prev[c], row_cur[c])
        if changed:
            mods.append({**{kk: k[i] for i, kk in enumerate(KEYS)}, **{f"{c}_old":v[0] for c,v in changed.items()},
                         **{f"{c}_new":v[1] for c,v in changed.items()}})
    modified = pd.DataFrame(mods)

    return {"added": added, "removed": removed, "modified": modified}

def diff_params(prev: dict|None, cur: dict):
    if not prev: 
        return {k: (None, cur[k]) for k in cur.keys()}
    out = {}
    for k, v in cur.items():
        pv = prev.get(k, None)
        if isinstance(v, (int,float)) and isinstance(pv, (int,float)):
            if not _is_close(pv, v):
                out[k] = (pv, v)
        else:
            if pv != v:
                out[k] = (pv, v)
    return out

def hash_legs(df: pd.DataFrame):
    d = _norm_keys(df)[list(KEYS + ("qty","vol","borrow"))].astype(str)
    key = sorted(d.agg("|".join, axis=1).tolist())
    return hashlib.sha256(json.dumps(key).encode()).hexdigest()[:10]

def log_change(path: str, payload: dict):
    with open(path, "a") as f:
        f.write(json.dumps({"ts": time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime()), **payload})+"\n")

def track_and_update(legs_cur: pd.DataFrame, params_cur: dict, log_path: str|None=None):
    """Compare against last snapshot, return (legs_diff, params_diff, meta), then update STATE."""
    legs_diff = diff_legs(STATE["legs_prev"], legs_cur)
    params_diff = diff_params(STATE["params_prev"], params_cur)

    meta = dict(step=STATE["step"]+1, legs_hash=hash_legs(legs_cur))
    if log_path:
        log_change(log_path, {"step": meta["step"], 
                              "legs_diff": {k: len(v) if isinstance(v, pd.DataFrame) else v for k,v in legs_diff.items()},
                              "params_diff": params_diff,
                              "legs_hash": meta["legs_hash"]})
    # update snapshot
    STATE["legs_prev"] = deepcopy(legs_cur)
    STATE["params_prev"] = deepcopy(params_cur)
    STATE["step"] += 1
    return legs_diff, params_diff, meta

In [21]:
import pandas as pd

options_df = pd.DataFrame({
    'mult':[1,1],
    'qty':[-100,100],
    'expiration': ['1/15/27','1/15/27'],
    'strike':[250.0,250.0],
    'spot':[252.29,252.29],
    'cp':['P','C'],
    'ea':['A','A'],
    'texp':[1.2416,1.2416],
    'vol':['27.45%','27.55%'],
    'borrow':[-0.006,-0.0060]
})

dividends_df = pd.DataFrame({
    'date':['11/8/25','2/8/26','5/10/26','8/9/26','11/8/26','2/8/27','5/8/27','8/8/27'],
    'amount':[0.26,0.26,0.26,0.26,0.26,0.40,0.40,0.40]
})

str_underlying='AAPL'
pd_val_date=pd.Timestamp('10/17/2025') 
combined_subset, combined, yc, mkt = run_pipeline(pd_val_date, options_df, dividends_df, "AAPL")
options_with_total = append_total_row(combined, multiplier=100.0)
options_with_total.columns
options_with_total[["expiration_x","strike","mdl_spot","cp","mdl_ea","mdl_texp","mdl_vol","mdl_borrow","mkt_bid","mdl_theo","mkt_ask"]] 

issues: []


/var/folders/tw/8dy4vcgj7h3fvzr8_jtkzs600000gn/T/ipykernel_2931/3142134925.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  out = pd.concat([df, pd.DataFrame([row])], ignore_index=True)


,expiration_x,strike,mdl_spot,cp,mdl_ea,mdl_texp,mdl_vol,mdl_borrow,mkt_bid,mdl_theo,mkt_ask
0,1/15/27,250.0,252.29,P,A,1.2416,27.45%,-0.006,24.6,24.855713,25.00
1,1/15/27,250.0,252.29,C,A,1.2416,27.55%,-0.006,36.7,37.380839,37.65
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.7,12.525126,13.05


In [27]:
# 1) define your inputs (you already have options_df, params, etc.)
import change_tracking as ct
params = {
    "underlying": str_underlying,
    "valuation_date": pd_val_date.strftime("%m/%d/%Y"),
    "r": 0.0308, "init_vol": 0.275, "mult": 100.0
}

# 2) run your pipeline (whatever you have now)
combined_subset, combined, yc, mkt = run_pipeline(pd_val_date, options_df, dividends_df, str_underlying)

# 3) append total if you like
options_with_total = append_total_row(combined, multiplier=params["mult"])

# 4) track changes between this run and the previous one
legs_changes, params_changes, meta = ct.track_and_update(options_df, params, log_path="cache/journal.jsonl")

print("Step:", meta["step"], "Legs hash:", meta["legs_hash"])
print("Added legs:", len(legs_changes["added"]), "Removed:", len(legs_changes["removed"]), 
      "Modified:", len(legs_changes["modified"]))
print("Param changes:", params_changes)


KeyboardInterrupt: 

In [ ]:
import change_tracking as ct


In [ ]:
track_and_update = ct.track_and_update


In [79]:
# usage: one row per trade_id
combined["qty"]=combined["mdl_mult"]

summary = summarize_trade_group(combined)
summary


qty_scale_base     100.000000
theo_unit           12.421265
delta_unit           1.031240
gamma_unit          -0.000519
vega_unit           -0.000429
theta_unit          -8.910794
mkt_unit_close      11.700000
mkt_unit_open       13.050000
theo_$            1242.126526
delta_raw          103.124040
gamma_raw           -0.051867
vega_raw            -0.042907
theta_raw         -891.079395
dtype: float64

In [80]:
import pandas as pd

class GreekUnits:
    """
    Describe how your current greeks are reported.
    We'll convert them to a standard:
      - per share
      - vega per 1 vol point (0.01)
      - theta per day
      - gamma per $ (i.e., Δ per $1 change)
    """
    def __init__(self,
                 per_contract: bool = False,   # True if greeks are per contract already
                 contract_multiplier: float = 100.0,
                 vega_per_vol_point: bool = True,  # True if vega is per 1 vol pt (0.01). False if per 1.0 (=100 pts)
                 theta_per_day: bool = True,       # True if theta is per day. False if per year.
                 gamma_per_dollar: bool = True):   # True if gamma is per $; False if per 1% (rare)
        self.per_contract = per_contract
        self.mult = contract_multiplier
        self.vega_per_vol_point = vega_per_vol_point
        self.theta_per_day = theta_per_day
        self.gamma_per_dollar = gamma_per_dollar

def standardize_greeks(df: pd.DataFrame,
                       units: GreekUnits,
                       prefix: str = "mdl_") -> pd.DataFrame:
    """
    Returns a copy with standardized greeks:
      mdl_delta_std, mdl_gamma_std, mdl_vega_std, mdl_theta_std
    All on a per-share basis, vega per 1 vol point (0.01), theta per day, gamma per $.
    If your later aggregation wants per-contract $, multiply by 'mult' at that step.
    """
    d = df.copy()

    # Scale from per-contract to per-share if needed
    per_contract_factor = (1.0 / units.mult) if units.per_contract else 1.0

    # Delta: usually already per share. Convert if per-contract.
    if f"{prefix}delta" in d:
        d[f"{prefix}delta_std"] = d[f"{prefix}delta"] * per_contract_factor

    # Gamma: keep as per $ (Δ per $1). Convert if per-contract.
    if f"{prefix}gamma" in d:
        d[f"{prefix}gamma_std"] = d[f"{prefix}gamma"] * per_contract_factor
        # If yours were per 1% (rare), convert to per $: gamma_% = dΔ/d(1%) = S * gamma$
        # then gamma$ = gamma_% / S
        # if not units.gamma_per_dollar and "spot" in d:
        #     d[f"{prefix}gamma_std"] = (d[f"{prefix}gamma_std"] / d["spot"].astype(float))

    # Vega: want per 1 vol point (0.01) per share.
    if f"{prefix}vega" in d:
        v = d[f"{prefix}vega"] * per_contract_factor
        # If source vega is per 1.0 (=100pts), convert to per 0.01: multiply by 100
        if not units.vega_per_vol_point:
            v = v * 100.0
        d[f"{prefix}vega_std"] = v

    # Theta: want per day per share.
    if f"{prefix}theta" in d:
        t = d[f"{prefix}theta"] * per_contract_factor
        # If theta is per year, convert to per day (≈ /365)
        if not units.theta_per_day:
            t = t / 365.0
        d[f"{prefix}theta_std"] = t

    # Theo: keep as price per share (model price). If per-contract, convert.
    if f"{prefix}theo" in d:
        d[f"{prefix}theo_std"] = d[f"{prefix}theo"] * per_contract_factor

    return d

In [83]:
u = GreekUnits(
    per_contract=False,          # greeks currently per share?
    contract_multiplier=100.0,
    vega_per_vol_point=True,     # vega already per 0.01?
    theta_per_day=True,          # theta already per day?
    gamma_per_dollar=True        # gamma per $?
)

d = standardize_greeks(combined, u, prefix="mdl_")

# Now aggregate using the *_std columns with your existing logic:
# e.g., nqty * mdl_theo_std, mdl_delta_std, mdl_gamma_std, mdl_vega_std, mdl_theta_std
d

,mdl_mult,expiration_x,strike,mdl_spot,cp,mdl_ea,mdl_texp,mdl_vol,mdl_borrow,mdl_rate,...,mkt_gamma,mkt_vega,mkt_underlying_price,mkt_optionId,qty,mdl_delta_std,mdl_gamma_std,mdl_vega_std,mdl_theta_std,mdl_theo_std
0,-100,1/15/27,250.0,252.29,P,A,1.2416,27.45%,-0.006,0.034684,...,0.005546,1.071017,252.29,134511704,-100,-0.396546,0.005454,1.069343,-8.152321,24.854431
1,100,1/15/27,250.0,252.29,C,A,1.2416,27.45%,-0.006,0.034684,...,0.004756,1.068378,252.29,134511703,100,0.634695,0.004935,1.068914,-17.063115,37.275696


In [35]:
from polygon.rest.models.summaries import SummaryResult


def summarize_straddle(df_rr):
    """
    Summarize the quote as a risk reversal. 
    assume the first option is a put and the second is a call.

    Args:
        df: DataFrame containing option data
    """
    # Calculate the risk reversal
    summarizable_fields=["theo","delta","ga","openInterest","delta","gamma","vega","underlying_price"]
    

In [44]:
combined.columns

Index(['mdl_mult', 'expiration_x', 'strike', 'mdl_spot', 'cp', 'mdl_ea',
       'mdl_texp', 'mdl_vol', 'mdl_borrow', 'mdl_rate', 'mdl_theo',
       'mdl_delta', 'mdl_gamma', 'mdl_theta', 'mdl_vega',
       'mdl_Combo_C_minus_P', 'mdl_ValuationDate', 'expiration_y', 'mkt_bid',
       'mkt_ask', 'mkt_iv', 'mkt_openInterest', 'mkt_volume', 'mkt_delta',
       'mkt_gamma', 'mkt_vega', 'mkt_underlying_price', 'mkt_optionId'],
      dtype='object')

## change detection logger: